<a href="https://www.kaggle.com/code/behnambaloochy/spaceship-titanic?scriptVersionId=142851750" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import make_pipeline

pipe_rf = make_pipeline(SimpleImputer(missing_values=np.nan, strategy="mean"), 
                        PCA(n_components=9), 
                        RandomForestClassifier(n_estimators=25, random_state=1, n_jobs=2))


# Data Preprocessing ----------------------------------------------------------------------------------
df_train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv', encoding='utf-8')
# print('train has {} rows and {} columns'.format(*df_train.shape))
# df_train.info()
# df_train[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = \
# df_train[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)
# df_train = df_train.dropna(how="any")

df_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv', encoding='utf-8')
# df_test.info()
# print('test has {} rows and {} columns'.format(*df_test.shape))
# df_test["CryoSleep"] = df_test["CryoSleep"].fillna(df_test["CryoSleep"].mean())


# df_test = df_test.fillna(0)
# print(df_train.head(10))
# print(df_test.head(10))

df_train["Transported"] = df_train["Transported"].astype(int)
# y_train = df_train[['Transported']].values
y_train = df_train.iloc[:, 13].values

# df_train['VIP'] = df_train['VIP'].astype(int)
# df_train['CryoSleep'] = df_train['CryoSleep'].astype(int)
X_train = df_train.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values

# Categorical Feature Encoding**********
# print('HomePlanet values: ', df_test["HomePlanet"].unique())
# options_HomePlanet = {label: idx for idx, label in enumerate(np.unique(df_train['HomePlanet']))}
options_HomePlanet = {}
for i in range(0, df_train["HomePlanet"].unique().shape[0]):
    options_HomePlanet[df_train["HomePlanet"].unique()[i]] = i+1
# df_train["HomePlanet"] = df_train["HomePlanet"].map(options_HomePlanet)
j = 0
for i in X_train[:,0]:
    X_train[j,0] = options_HomePlanet.get(i)
    j += 1

# print('Train Cabin values: ', df_train[3].unique())
# print('Test Cabin values: ', df_test["Cabin"].unique())
options_Cabin = {}
for i in range(0, df_train["Cabin"].unique().shape[0]):
    options_Cabin[df_train["Cabin"].unique()[i]] = i+1
j = 0
for i in X_train[:,2]:
    X_train[j,2] = options_Cabin.get(i)
    j += 1

# print('Destination values: ', df_test["Destination"].unique())
options_Destination = {}
for i in range(0, df_train["Destination"].unique().shape[0]):
    options_Destination[df_train["Destination"].unique()[i]] = i+1
j = 0
for i in X_train[:,3]:
    X_train[j,3] = options_Destination.get(i)
    j += 1
# **********

X_train[:,1] = np.where(X_train[:,1] == 'True', 1, 0)
X_train[:,5] = np.where(X_train[:,5] == 'True', 1, 0)
X_train = np.float_(X_train)
# print('Class labels:', np.unique(y_train))
# imr = imr.fit(X_train)
# X_train = imr.transform(X_train)

# df_test['VIP'] = df_test['VIP'].astype(int)
# df_test['CryoSleep'] = df_test['CryoSleep'].astype(int)
X_test = df_test.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values

# Categorical Feature Encoding**********
j = 0
for i in X_test[:,0]:
    X_test[j,0] = options_HomePlanet.get(i)
    j += 1

j = 0
for i in X_test[:,2]:
    X_test[j,2] = options_Cabin.get(i)
    j += 1

j = 0
for i in X_test[:,3]:
    X_test[j,3] = options_Destination.get(i)
    j += 1
# **********

X_test[:,1] = np.where(X_test[:,1] == 'True', 1, 0)
X_test[:,5] = np.where(X_test[:,5] == 'True', 1, 0)
X_test = np.float_(X_test)

# imr = imr.fit(X_test)
# X_test = imr.transform(X_test)

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=1, stratify=y_train)

# Feature Scaling---------------------------------------------------------------------------------------
# sc = StandardScaler()
# sc.fit(X_train)
# X_train_std = sc.transform(X_train)
# X_test_std = sc.transform(X_test)

# sklearn-Random Forest Training------------------------------------------------------------------------
# forest = RandomForestClassifier(n_estimators=25, random_state=1, n_jobs=2)
# forest.fit(X_train, y_train)
pipe_rf.fit(X_train, y_train)
# sklearn-Random Forest Prediction----------------------------------------------------------------------
y_pred = pipe_rf.predict(X_test)

# Submission--------------------------------------------------------------------------------------------
trans = []
for i in y_pred:
    trans.append(i==1)
submission = np.column_stack((df_test.PassengerId, trans))
print(submission)
print(np.shape(submission))
df_result = pd.DataFrame(submission)
df_result.to_csv("/kaggle/working/submission.csv", header=['PassengerId', 'Transported'], index=False)

# feature_names = ['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'Shopping', 'Spa', 'VRDeck']
